In [ ]:
!pip install -r requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! ls /content/gdrive/My\ Drive/arafat/features_train.pickle

'/content/gdrive/My Drive/arafat/features_train.pickle'


In [ ]:
import pickle
path_features_train = "/content/gdrive/My Drive/arafat/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

In [ ]:
path_labels_train = "/content/gdrive/My Drive/arafat/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

In [ ]:
path_features_test = "/content/gdrive/My Drive/arafat/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

In [ ]:
path_labels_test = "/content/gdrive/My Drive/arafat/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)

In [ ]:
print(features_train.shape)
print(features_test.shape)
print(labels_train.shape)
print(labels_test.shape)

(57207, 5000)
(6357, 5000)
(57207,)
(6357,)


In [ ]:
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# svc_0 =svm.SVC()

# print('Parameters currently in use:\n')
# print(svc_0.get_params())

In [ ]:
# # C
# C = [.001, .01,.1,.5,1]

# # gamma
# gamma = [.01, .1, 1]

# # degree
# degree = [1,2,3]

# # kernel
# kernel = ['linear']

# # probability

# # Create the random grid
# random_grid = {'C': C,
#               'kernel': kernel,
#               'gamma': gamma,
#               'degree': degree
#              }

# print(random_grid)

In [ ]:
# # First create the base model to tune
# svc = svm.SVC(random_state=8)

# # Definition of the random search
# random_search = RandomizedSearchCV(estimator=svc,
#                                    param_distributions=random_grid,
#                                    n_iter=1,
#                                    scoring='accuracy',
#                                    cv=3, 
#                                    verbose=1, 
#                                    random_state=8)

# # Fit the random search model
# random_search.fit(features_train, labels_train)

In [ ]:
# print("The best hyperparameters from Random Search are:")
# print(random_search.best_params_)
# print("")
# print("The mean accuracy of a model with these hyperparameters is:")
# print(random_search.best_score_)

Grid Search cross validation

In [ ]:
# # Create the parameter grid based on the results of random search 
# C = [float(x) for x in np.linspace(start = 0.5, stop = 1, num = 5)]


# param_grid = [
#   {'C': C}
# ]

# # Create a base model
# svc = svm.SVC()

# # Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
# # cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# # Instantiate the grid search model
# grid_search = GridSearchCV(estimator=svc, 
#                            param_grid=param_grid,
#                            scoring='accuracy',
#                            cv = 3,
#                            verbose=1)

# # Fit the grid search to the data
# grid_search.fit(features_train, labels_train)

In [ ]:
#ADABOOST Classifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.01,0.05,0.1,0.3,1],
 'loss' : ['linear', 'square', 'exponential']
 }

grid_search = RandomizedSearchCV(AdaBoostRegressor(),
 param_distributions = param_dist,
 cv=3,
 n_iter = 10,
 n_jobs=-1)

grid_search.fit(features_train, labels_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=AdaBoostRegressor(base_estimator=None,
                                               learning_rate=1.0, loss='linear',
                                               n_estimators=50,
                                               random_state=None),
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.3,
                                                          1],
                                        'loss': ['linear', 'square',
                                                 'exponential'],
                                        'n_estimators': [50, 100]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [ ]:
print("The best hyperparameters from Grid Search are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

The best hyperparameters from Grid Search are:
{'C': 1.0}

The mean accuracy of a model with these hyperparameters is:
0.8035555089412135


In [ ]:
best_svc = grid_search.best_estimator_

In [ ]:
best_svc

AdaBoostRegressor(base_estimator=None, learning_rate=0.3, loss='exponential',
                  n_estimators=100, random_state=None)

In [ ]:

best_svc.fit(features_train, labels_train)

SVC()

In [ ]:
svc_pred = best_svc.predict(features_test)

In [ ]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(labels_train, best_svc.predict(features_train)))

The training accuracy is: 
0.812138374674428


In [ ]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(labels_test, svc_pred))

The test accuracy is: 
0.8002202296680825


In [ ]:

# Classification report
print("Classification report")
print(classification_report(labels_test,svc_pred))

Classification report
              precision    recall  f1-score   support

           0       0.75      0.92      0.82      3200
           1       0.89      0.68      0.77      3157

    accuracy                           0.80      6357
   macro avg       0.82      0.80      0.80      6357
weighted avg       0.82      0.80      0.80      6357



In [ ]:
base_model = svm.SVC()
base_model.fit(features_train, labels_train)
accuracy_score(labels_test, base_model.predict(features_test))

0.8002202296680825

In [ ]:

best_svc.fit(features_train, labels_train)
accuracy_score(labels_test, best_svc.predict(features_test))

0.8002202296680825

In [ ]:
d = {
     'Model': 'SVM',
     'Training Set Accuracy': accuracy_score(labels_train, best_svc.predict(features_train)),
     'Test Set Accuracy': accuracy_score(labels_test, svc_pred)
}

df_models_svc = pd.DataFrame(d, index=[0])

In [ ]:
df_models_svc

,Model,Training Set Accuracy,Test Set Accuracy
0,SVM,0.812138,0.80022


In [ ]:
with open('/content/gdrive/My Drive/colab_work/best_svc.pickle', 'wb') as output:
    pickle.dump(best_svc, output)
    
with open('/content/gdrive/My Drive/colab_work/df_models_svc.pickle', 'wb') as output:
    pickle.dump(df_models_svc, output)